# Prueba

Utilizar `microsoft/phi-2` para crear un chatbot.

## Start by loading the model

We have to start by getting the model and doing a test run. We can check the huggingface [`microsoft/phi-2` page](https://huggingface.co/microsoft/phi-2) to check info on the model.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

#model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


/home/grego/code/jobsearch/ipglobal/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.16s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


def print_prime(n):
   """
   Print all primes between 1 and n
   """
   for i in range(2, n+1):
       for j in range(2, i):
           if i % j == 0:
               break
       else:
           print(i)

print_prime(20)
```

## Exercises

1. Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.

```python
def sum_even(numbers):
    """
    Returns the sum of all even numbers in the list
    """
    return sum(filter(lambda x: x % 2 == 0, numbers))

print(sum_even([1, 2, 3, 4, 5, 6])) # Output: 12
```

2. Write a Python function that takes


## Experiment with the model

After ensuring we can properly run the model, we can start trying out some things to get a sense of how the model performs.

In [2]:
def run_prompt(prompt):
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=False)
        outputs = model.generate(**inputs, max_length=200)
        text = tokenizer.batch_decode(outputs)[0]
    return text

In [3]:
print(run_prompt("""
The following is a friendly chat between Bob and Alice.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: 
""".strip()))

The following is a friendly chat between Bob and Alice.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Try to study in a quiet environment, free from distractions.
Alice: I will try that.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Try to study in a quiet environment, free from distractions.
Alice: I will try that.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Try to study in a quiet environment, free from distractions.
Alice: I will try that.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to


### Learned fact 1

We have to be carefull when formatting the prompt to `phi-2`. It can be very sensitive to the way you write your instructions.

For example, the following prompt

```python
prompt = """
The following is a friendly chat between Bob and Alice.
Bob: I will provide clear and concise answers.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: 
"""
```

gives a very different result if you run it straight away `run_prompt(prompt)` or if you strip out the start and end linebreaks `run_prompt(prompt.strip())`. You can check the differences.

In [9]:
chat_prompt = """
The following is a friendly chat between Bob and Alice.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob:"""

In [10]:
# when you keep the breaklines, it spits some weird text, basically totally ignoring the chat indications
print(run_prompt(chat_prompt))


The following is a friendly chat between Bob and Alice.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Have you tried using a timer? It can help you stay on track.
Alice: How does that work?
Bob: Well, you set the timer for a specific amount of time, like 25 minutes, and focus solely on studying during that time. Then, you take a short break before starting the timer again.
Alice: That sounds interesting. How long should the breaks be?
Bob: It's recommended to have a 5-minute break after every 25 minutes of studying. This way, you can recharge and maintain your focus.
Alice: I'll give it a try. Thanks for the suggestion, Bob!
Bob: You're welcome, Alice. Let me know if it helps.

The following is a conversation between Sarah and John.
Sarah: I'm having trouble understanding the


In [6]:
# when you strip away the braklines, you get more or less the intended behaviour
print(run_prompt(chat_prompt.strip()))

The following is a friendly chat between Bob and Alice.
Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestions?
Bob: Have you tried using a timer? It's a great way to manage your time effectively.
Alice: How does it work?
Bob: Well, you set the timer for a specific amount of time, like 25 minutes, and focus solely on studying during that time. After the timer goes off, take a short break before starting the next 25-minute study session.
Alice: That sounds interesting. How long should the breaks be?
Bob: It's recommended to have a 5-minute break after every 25 minutes of studying. This allows your brain to rest and recharge.
Alice: I'll give it a try. Thanks for the suggestion, Bob!
Bob: You're welcome, Alice. I hope it helps you stay focused and productive.

The following is a conversation between Sarah and John


---

In [7]:
print(run_prompt("""
{"name":"Alice","surname":"Johnson"}
The follwing is a html table that uses the variables in the json above. After the table there's DONE:
<table>
""".strip()))

{"name":"Alice","surname":"Johnson"}
The follwing is a html table that uses the variables in the json above. After the table there's DONE:
<table>
<tr>
<td>Name</td>
<td>Surname</td>
</tr>
<tr>
<td>Alice</td>
<td>Johnson</td>
</tr>
</table>

A:

You can use the following code:
import json

json_data = '{"name":"Alice","surname":"Johnson"}'

data = json.loads(json_data)

print(data['name'])
print(data['surname'])

A:

You can use json.loads() to convert the json string to a python dictionary.
import json

json_data = '{"name":"Alice","surn


In [8]:
print(run_prompt("""
Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output:
"""))


Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output:
Mathematics is like a lighthouse, guiding us through the vast ocean of knowledge. Just as a lighthouse emits a steady beam of light to illuminate the way for ships, mathematics provides us with a framework to understand and navigate the complexities of the world.

Similar to how a lighthouse helps ships avoid dangerous rocks and navigate safely, mathematics equips us with problem-solving skills and logical reasoning to overcome challenges and make informed decisions. It serves as a beacon of clarity, shedding light on abstract concepts and enabling us to see patterns and connections that may not be immediately apparent.

Furthermore, just as a lighthouse stands tall and sturdy, mathematics provides a solid foundation for various disciplines and fields of study. It serves as a common language that transcends cultural and linguistic barriers, allowing scientists, engineers, and researchers to communicate an